# Surprise homework

In [1]:
import io
import pandas as pd
from collections import defaultdict

### Functions

In [2]:
from surprise import Dataset
from surprise import NormalPredictor
from surprise import SVD
from surprise import KNNWithMeans
from surprise import get_dataset_dir
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

### Load data

In [3]:
k = 30
n = 5
threshold = 3.52
measure = ['RMSE']
test_rmse = 'test_rmse'

In [4]:
data = Dataset.load_builtin('ml-100k')
data

In [5]:
trainset, testset = train_test_split(data, test_size=.25)

### Define algorithms

In [6]:
define_algorithms = {
    'NP': NormalPredictor(),
    'KNN_cosine': KNNWithMeans(k = k, sim_options = { 'name': 'cosine' }),
    'KNN_MSD': KNNWithMeans(k = k),
    'KNN_pearson': KNNWithMeans(k = k, sim_options = { 'name': 'pearson' }),
    'svd' : SVD()
}

In [7]:
def crv_rsma(define_algorithms, data, measure, test_rmse):
    rsma = {}
    for [name, algorithm] in define_algorithms.items():
        crv = cross_validate(algorithm, data, measures = measure, verbose=True)
        rsma[name] = round(crv[test_rmse].mean(), 3)
    return rsma

rsma = crv_rsma(define_algorithms, data, measure, test_rmse)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5198  1.5234  1.5316  1.5048  1.5126  1.5184  0.0092  
Fit time          0.09    0.12    0.12    0.12    0.12    0.11    0.01    
Test time         0.16    0.10    0.15    0.15    0.10    0.13    0.03    
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9443  0.9731  0.9587  0.9622  0.9594  0.9595  0.0092  
Fit time          0.73    0.73    0.74    0.73    0

In [8]:
rsma

{'NP': 1.518,
 'KNN_cosine': 0.96,
 'KNN_MSD': 0.953,
 'KNN_pearson': 0.951,
 'svd': 0.936}

In [9]:
bestalgo_name = min(rsma.items(), key=lambda x: x[1])[0]
bestalgo = define_algorithms[bestalgo_name]
bestalgo_name

'svd'

### Calculate precision@k and recall@k

In [10]:
def precision_recall_at_k(predictions, k = 10, threshold = 3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions, recalls

In [11]:
bestalgo.fit(trainset)
predictions = bestalgo.test(testset)
precisions, recalls = precision_recall_at_k(predictions, k = 5, threshold = threshold)

In [12]:
precisionak = round(sum(precision for precision in precisions.values()) / len(precisions), 3)
precisionak

0.731

In [13]:
recallak = round(sum(recall for recall in recalls.values()) / len(recalls), 3)
recallak

0.362

### Predict

In [14]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [15]:
def recommended_items(top_n, user):
    for uid, user_ratings in top_n.items():
        if uid == user: 
            items = [iid for (iid, _) in user_ratings]
            rate = user_ratings
            break
    return rate, items

In [16]:
def items_info(items):
    data_path = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    info = {}
    data = pd.read_csv(data_path, sep='|',encoding='ISO-8859-1', header = None) 
    for i in items:
        row = data.iloc[int(i)]
        info[i] = (row[1], row[2])
    return info

In [17]:
testset = trainset.build_anti_testset()
predictions = bestalgo.test(testset)
top_n = get_top_n(predictions, n)

user = '20'
rate, films = recommended_items(top_n, user)
rate=dict(rate)
films

['189', '483', '165', '169', '15']

In [18]:
info = items_info(films)
info

{'189': ('Henry V (1989)', '01-Jan-1989'),
 '483': ('Maltese Falcon, The (1941)', '01-Jan-1941'),
 '165': ('Manon of the Spring (Manon des sources) (1986)', '01-Jan-1986'),
 '169': ('Cinema Paradiso (1988)', '01-Jan-1988'),
 '15': ('French Twist (Gazon maudit) (1995)', '01-Jan-1995')}

In [19]:
def writing_to_file(user, info, rate):
    file = open('recsys_2_2.txt', 'w')
    file.write('User '+ user + '\n')
    for i in info:
        file.write(str(i) + ' ' + str(info[i]) + ' ' + str(round(rate[i],3)) + '\n')
    file.close()
    
writing_to_file(user, info, rate)